In [ ]:
import pandas as pd
import sqlite3


df = pd.read_csv('/Users/USERS_NAME/Desktop/bilibili_knowledge_rank_20260127.csv') 

conn = sqlite3.connect(':memory:')
df.to_sql('bili_videos', conn, index=False, if_exists='replace')

sql_query = """
SELECT 
    CASE 
        WHEN duration_sec < 180 THEN 'Short (<3m)'
        WHEN duration_sec BETWEEN 180 AND 600 THEN 'Medium (3-10m)'
        ELSE 'Long (>10m)'
    END AS video_type,
    
    COUNT(*) as video_count,
    
    -- Calculate Average Views
    ROUND(AVG(views), 0) as avg_views,
    
    -- Calculate "Coin Rate" (Coins per 1k views) - The "Quality" Metric
    ROUND(AVG(CAST(coin AS FLOAT) / views) * 1000, 2) as coins_per_1k_views,
    
    -- Calculate "Virality" (Shares per 1k views)
    ROUND(AVG(CAST(share AS FLOAT) / views) * 1000, 2) as shares_per_1k_views

FROM bili_videos
GROUP BY video_type
ORDER BY coins_per_1k_views DESC;
"""

results = pd.read_sql_query(sql_query, conn)
print("📊 Bilibili Insight Report:")
print(results)

📊 Bilibili Insight Report:
       video_type  video_count  avg_views  coins_per_1k_views  \
0     Long (>10m)           34  1828689.0               12.72   
1  Medium (3-10m)           29  2465137.0                4.42   
2     Short (<3m)           33  2039254.0                3.22   

   shares_per_1k_views  
0                 1.81  
1                 1.92  
2                 1.52  
